# Lour's Pork Barrel Classifier (羅老師肉桶文本分類器)🐖
## Convolutional Neural Networks for Pork Barrel Project Classification 

-------------------------

### Stage 1: Libaries & Dependencies

In [1]:
# built-in library
import math
import re
import collections
import zipfile
import random
from itertools import chain

# ML & Deep Learning/ NLP toolkit
import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#from keras.layers import Flatten

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

-------------------------

### Stage 2: Data Preprocessing (Training Data: Introduction of Bills and Legislation from 6th Session to 7th Session, 2004-2012)

#### (1) Read file 

In [2]:
# read file
df = pd.read_csv('data/Pork Bill - 2021-05-20.csv',encoding='utf-8')

# combine abstract of bill and title 
df['text'] =  df['Title'] + df['Content'].fillna(df['Title'])

# drop conten without having any characters
# view na's row: df[df['text'].isnull()==True]
data = df[['text', 'pork_bill']].dropna(subset=['text'])

In [3]:
print(" Pork Legislation", data['pork_bill'].value_counts()[1],'\n', 
      "None-Pork Legislation", data['pork_bill'].value_counts()[0])

 Pork Legislation 2510 
 None-Pork Legislation 4733


#### (2) Balancing the Training Samples

random.seed(24)
data = data[data['pork_bill']==1].append(data[data['pork_bill']==0].reset_index(drop=True).loc[[random.randint(0, 2500) for i in range(2500)]]).reset_index(drop=True)

print(" Pork Legislation", data['pork_bill'].value_counts()[1],'\n', 
      "None-Pork Legislation", data['pork_bill'].value_counts()[0])

#### (2) Tokenization

In [4]:
import collections
import numpy as np
import jieba
from itertools import chain


def jieba_cut(filename):
    """
    cut Chinese and remove stop words
    Reference: https://www.cnblogs.com/Luv-GEM/p/10836454.html
    Stopwords: https://www.kaggle.com/rikdifos/english-and-chinese-stopwords?select=cn_stopwords.txt
    """
    stop_list = [i.strip() for i in open('cn_stopwords.txt','r',encoding='utf-8')]  
    news_cut = []
    news_list = []
    for line in filename:    
        if line:
            news_cut = list(jieba.cut(''.join(line),cut_all=False,HMM=True))  
            news_list.append([word.strip() for word in news_cut if word not in stop_list and len(word.strip())>0]) 
    news_list = list(chain.from_iterable(news_list))  
    return news_list

def clearPucts(context):
    """
    remove punctuation
    ref: https://chenyuzuoo.github.io/posts/28001/
    """
    context = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "", context)
    context = re.sub("[【】╮╯▽╰╭★→「」]+","", context)
    context = re.sub("！，❤。～《》：（）【】「」？”“；：、","",context)
    context = re.sub("\s","",context)
    return context

def seg_char(sent):
    """
    cut Chinese and remove stop words
    ref: https://blog.csdn.net/renyuanfang/article/details/86487367
    """
    # split
    pattern_char_1 = re.compile(r'([\W])')
    parts = pattern_char_1.split(sent)
    parts = [p for p in parts if len(p.strip())>0]
    # cut sentence
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = pattern.split(sent)
    chars = [w for w in chars if len(w.strip())>0]
    chars = ' '.join(chars)
    return chars

In [5]:
data_clean = [seg_char(text) for text in [clearPucts(text) for text in data.text]]

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**18)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

#### (3) Padding

In [6]:
MAX_LEN = max([len(sentence) for sentence in data_clean])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)


#### (4) Spliting Training/ Testing Set

In [7]:
data_labels = data.pork_bill.values

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    data_inputs, data_labels, test_size=0.33, random_state=42)

print("Shape of X Train:", train_inputs.shape, '\n'
      "Shape of X Test :", test_inputs.shape,'\n'
      "Shape of Y Trian:", train_labels.shape , '\n'
      "Shape of Y Test :", test_labels.shape )

Shape of X Train: (4852, 785) 
Shape of X Test : (2391, 785) 
Shape of Y Trian: (4852,) 
Shape of Y Test : (2391,)


In [9]:
unique_train_labels, counts_train_labels = np.unique(train_labels, return_counts=True)

print(' Num of Train Set:' , train_inputs.shape[0], '\n'
      ,'Not Pork vs Pork:', dict(zip(unique_train_labels, counts_train_labels)))

 Num of Train Set: 4852 
 Not Pork vs Pork: {0: 3167, 1: 1685}


In [10]:
unique_test_labels, counts_test_labels = np.unique(test_labels, return_counts=True)

print(' Num of Test Set:' , test_inputs.shape[0], '\n'
      ,'Not Pork vs Pork:', dict(zip(unique_test_labels, counts_test_labels)))

 Num of Test Set: 2391 
 Not Pork vs Pork: {0: 1566, 1: 825}


-------------------------


### Stage 3: Model and Building

#### (1) Using the Subclassing API to Build Dynamic Model

In [11]:
class PorkCNN(tf.keras.Model):
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=100,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(PorkCNN, self).__init__(name=name)
        self.embedding=layers.Embedding(vocab_size, emb_dim, input_length=30, input_shape=(BATCH_SIZE, 30))
        self.bigram=layers.Conv1D(filters=nb_filters, kernel_size=2, strides = 2, padding="valid", activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,kernel_size=3, strides = 2, padding="valid",activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,kernel_size=4, strides = 1, padding="valid",activation="relu")
        self.fivegram = layers.Conv1D(filters=nb_filters,kernel_size=5,strides = 1, padding="valid",activation="relu")
        self.sixgram = layers.Conv1D(filters=nb_filters,kernel_size=6,strides = 3, padding="valid",activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.last_dense = layers.Dense(units=1, activation="sigmoid") 
        #self.flaten =  layers.Flatten()
        #self.batch_nor = layers.BatchNormalization()
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        x_4 = self.fourgram(x)
        x_4 = self.pool(x_4) 
        x_5 = self.fivegram(x)
        x_5 = self.pool(x_5)
        x_6 = self.sixgram(x)
        x_6 = self.pool(x_6)          
        merged = tf.concat([x_1, x_2, x_3, x_4, x_5, x_6], axis=-1) 
        #merged = self.batch_nor(merged)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        #merged = self.flaten(merged)
        output = self.last_dense(merged)        
        return output

In [12]:
VOCAB_SIZE = tokenizer.vocab_size  # 5000 tokenizer.vocab_size
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2 #len(set(train_labels))
DROPOUT_RATE = 0.4
BATCH_SIZE = 230
NB_EPOCHS = 80

Dcnn = PorkCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

#### (2) Compile and Summary of the Model

In [13]:
Dcnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
Dcnn.build(input_shape = (BATCH_SIZE, 30)) # (BATCH_SIZE, input_length=30)
Dcnn.summary()

Model: "dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  586600    
_________________________________________________________________
conv1d (Conv1D)              multiple                  40100     
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  60100     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  80100     
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  100100    
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  120100    
_________________________________________________________________
global_max_pooling1d (Global multiple                  0      

#### (3) Check Point Path

In [14]:
checkpoint_path = "checkpoint_recode/ckpt_manager"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=10)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Checkpoint Located!!")

Checkpoint Located!!


-------------------------

### Stage 4: Train the Model

In [15]:
early_stop = EarlyStopping(monitor='val_loss',patience=5)

In [16]:
Dcnn.fit(train_inputs, train_labels,
         validation_data=(test_inputs, test_labels),
         batch_size=BATCH_SIZE,
         #validation_split=0.1,
         epochs=NB_EPOCHS,
         callbacks=[early_stop])
ckpt_manager.save()

Epoch 1/80
22/22 [==============================] - 49s 2s/step - loss: 0.0429 - accuracy: 0.9858 - val_loss: 0.1884 - val_accuracy: 0.9469
Epoch 2/80
22/22 [==============================] - 46s 2s/step - loss: 0.0431 - accuracy: 0.9858 - val_loss: 0.1825 - val_accuracy: 0.9473
Epoch 3/80
22/22 [==============================] - 43s 2s/step - loss: 0.0407 - accuracy: 0.9878 - val_loss: 0.1726 - val_accuracy: 0.9506
Epoch 4/80
22/22 [==============================] - 48s 2s/step - loss: 0.0440 - accuracy: 0.9854 - val_loss: 0.1781 - val_accuracy: 0.9477
Epoch 5/80
22/22 [==============================] - 50s 2s/step - loss: 0.0454 - accuracy: 0.9868 - val_loss: 0.1724 - val_accuracy: 0.9460
Epoch 6/80
22/22 [==============================] - 49s 2s/step - loss: 0.0418 - accuracy: 0.9864 - val_loss: 0.1770 - val_accuracy: 0.9473
Epoch 7/80
22/22 [==============================] - 49s 2s/step - loss: 0.0426 - accuracy: 0.9862 - val_loss: 0.1905 - val_accuracy: 0.9431
Epoch 8/80
22/22 [==

'checkpoint_recode/ckpt_manager/ckpt-2'

-------------------------

### Stage 5: Evaluation


In [31]:
losses = pd.DataFrame(Dcnn.history.history)
losses.head()

Empty DataFrame
Columns: []
Index: []

In [32]:
losses[['accuracy','val_accuracy']].plot()
losses[['loss','val_loss']].plot()

KeyError: "None of [Index(['accuracy', 'val_accuracy'], dtype='object')] are in the [columns]"

In [34]:
losses.plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.xlabel("Learning Curves: the mean training loss and accuracy measured over each epochs")
#plt.savefig('images/learning_curves.png')
plt.show()


TypeError: no numeric data to plot

In [20]:
from sklearn.metrics import classification_report,confusion_matrix

In [21]:
evaluation_model = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(evaluation_model)

11/11 [==============================] - 5s 486ms/step - loss: 0.1816 - accuracy: 0.9460
[0.18157580494880676, 0.9460476636886597]


In [22]:
# pd.DataFrame(confusion_matrix(test_labels,predictions))
predictions = Dcnn.predict(test_inputs)
predictions = np.where(predictions >0.8 , 1, 0)
print(classification_report(test_labels, predictions))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1566
           1       0.94      0.90      0.92       825

    accuracy                           0.95      2391
   macro avg       0.94      0.94      0.94      2391
weighted avg       0.95      0.95      0.95      2391



In [23]:
t = pd.DataFrame(confusion_matrix(test_labels,predictions), 
             columns=['Predictions: Not Pork(0)','Predictions:Pork(1)'])
t.index = ['Acutal: Not Pork(0)', 'Acutal: Pork (1)']
t

Predictions: Not Pork(0)  Predictions:Pork(1)
Acutal: Not Pork(0)                      1517                   49
Acutal: Pork (1)                           81                  744

-------------------------

### Stage 6: Try the Model with New Data and Export an End-to-end Model


#### (1) Test New Dataset from 2000 samples from Legislative Questions from 6th Session 

In [24]:
def as_num(x):
    """
    keep 10 decimals
    """
    y = '{:.15f}'.format(x) 
    return y

In [25]:
import pandas as pd
import random

# test the trined model using new text from 200 samples from Leislative Questions
LQ6 = pd.read_csv('data/LQ_6th.csv')

# drop '0': legislor  never have proposed any legisaltive questions. 
LQ6 = LQ6[LQ6['title'].isin(['0'])== False]

# select text with number of charecters within EMB_DIM; len(sub_set) = 11162
# sample_df = LQ6.loc[[len(char_num) for char_num in LQ6.title if len(char_num) < MAX_LEN], ['legislator','title', 'topic', 'category', 'keywords', 'ques_type']].reset_index(drop=True)
random.seed(42)
sample_df = LQ6.iloc[random.sample(range(len(LQ6)), 2000)][['legislator', 'title', 'topic', 'category', 'keywords', 'ques_type']]
sub_set = [seg_char(text) for text in [clearPucts(text) for text in sample_df.title]]

##### Top 10 of 2000 Samples

In [26]:
pd.DataFrame(
    {'legislator': sample_df.legislator,
     'Pork Value(Constituency Interest)':[as_num(Dcnn(np.array([tokenizer.encode(line)]), training=True).numpy()[0][0]) for line in sub_set],
     'Legislative Questions ': sample_df.title,
     'Topic': sample_df.topic,
     'Key Word':sample_df.keywords}).sort_values(by=['Pork Value(Constituency Interest)'], ascending = False).head(15)

legislator Pork Value(Constituency Interest)  \
4546        林鴻池                 0.997251331806183   
1761        彭添富                 0.997249662876129   
9892        陳啟昱                 0.993460297584534   
35          丁守中                 0.992445349693298   
9699        鄭運鵬                 0.992028832435608   
1641        彭添富                 0.989451169967651   
8531         薛凌                 0.985388755798340   
1706        彭添富                 0.984269261360168   
2282        曾華德                 0.981687068939209   
5342        沈智慧                 0.978373408317566   
2245        曾燦燈                 0.978322029113770   
8335    蔡煌瑯(離職)                 0.977264642715454   
3304        林建榮                 0.975652098655701   
7382        盧秀燕                 0.975268840789795   
4435        林重謨                 0.975120842456818   

                                 Legislative Questions              Topic  \
4546  針對諸多已獲得五年五百億與卓越計畫獎補助的大學擬調漲學雜費，但教育部長杜正勝曾承諾，獲得上述...  教育補助 ; 教育費用 ; 大學   
1761        針對「辦理九十四年原住民中低收入戶家庭租屋補助計畫」專案補助計畫，特向行政院提出質詢。            原住民族生活   
9892  鑑於現行《所得稅法》第十七條規定特別扣除額教育支出部分，僅以納稅義務人之子女就讀大專院校為限...  所得稅 ; 教育費用 ; 扣除額   
35    針就民眾陳情指出，目前政府對身心障礙者提供之生活津貼，依身心障礙程度等級分為1000元至50...           身心障礙者福利   
9699  為教育部因編列退撫經費、就學貸款利息補貼、學雜費減免補貼，而排擠國立高中職預算，就目前教育資...  教育補助 ; 高中 ; 教育經費   
1641                      針對「豪雨成災，農作物損失補償」問題，特向行政院提出質詢。              農業補助   
8531  針對雲林縣政府擬訂「雲林縣地方稅徵收自治條例」草案，開始對煉油廠及煉鋼鐵廠開徵「綠稅」。本席...               地方稅   
1706               針對「觀音鄉保生社區風貌營造規劃設計」專案補助計畫，特向行政院提出質詢。       社區計畫 ; 政府補助   
2282  為民國38年至43年間戌守大陳島等地區之中華民國前江、浙、閩、粵反共救國軍補發薪餉問題，攸關...                軍餉   
5342  針對內政部公布之人口統計資料顯示，去年全年台灣地區新生兒數目為二十一萬六千餘人，比前年再減少...        出生率 ; 人口政策   
2245  鑑於國內航空業者，以油價上漲為由，申請向消費者額外徵收每一航段七美元的油料附加費，嚴重影響消...         民航 ; 石油價格   
8335  請中央惠予補助辦理「九二一震災出租先租後售及救濟性住宅（平價住宅）」之管理維護費用，特向行政...       災害救助 ; 政府補助   
3304  為宜蘭縣冬山鄉南富段土地因鄰近冬山河，遭100米風景線逕為分割，致原符合農發條例規定可興建農...  農地 ; 農村住宅 ; 政府補助   
7382  針對早期退除役軍官給與補助金發放金額過低，實無法解決終身生活所需，希望相關單位考量實際情況，...              榮民福利   
4435                    針對彰化縣教育經費不足，急需中央補助，特向行政院提出緊急質詢。       地方財政 ; 教育經費   

                       Key Word  
4546             卓越計畫獎補助 ; 大學學費  
1761            原住民中低收入戶 ; 租屋補助  
9892        所得稅法 ; 特別扣除額 ; 教育支出  
35                    身心障礙者生活津貼  
9699  退撫經費 ; 就學貸款利息補貼 ; 學雜費減免補貼  
1641                   豪雨 ; 農作物  
8531         地方稅 ; 綠稅 ; 空氣汙染防治法  
1706                       保生社區  
2282                  反共救國軍補發薪餉  
5342          人口統計 ; 新生兒 ; 出生育率  
2245                 航空業 ; 油價上漲  
8335         九二一 ; 救濟性住宅 ; 平價住宅  
3304            冬山鄉 ; 農舍 ; 徵收補償  
7382               早期退除役軍官給與補助金  
4435                 彰化縣 ; 教育經費

##### Last 10 Rows of 2000 Samples

In [27]:
pd.DataFrame(
    {'legislator': sample_df.legislator,
     'Pork Value(Constituency Interest)':[as_num(Dcnn(np.array([tokenizer.encode(line)]), training=True).numpy()[0][0]) for line in sub_set],
     'Legislative Questions ': sample_df.title,
     'Topic': sample_df.topic,
     'Key Word':sample_df.keywords}).sort_values(by=['Pork Value(Constituency Interest)'], ascending = False).tail(15)

legislator Pork Value(Constituency Interest)  \
1169         孫大千                 0.000005136100754   
845          吳清池                 0.000004724090559   
11021        黃昭順                 0.000004709134373   
4356         林進興                 0.000004578928838   
825          吳清池                 0.000004556273325   
1504         張顯耀                 0.000004225719749   
6636         王幸男                 0.000003777578740   
5603         潘孟安                 0.000003524285376   
9039          邱毅                 0.000003422539294   
3525         林惠官                 0.000002675824362   
6532         王幸男                 0.000001750110300   
5595         潘孟安                 0.000000948772026   
716          吳敦義                 0.000000801520287   
2339         朱俊曉                 0.000000790882439   
8839         賴清德                 0.000000279684286   

                                  Legislative Questions               Topic  \
1169     針對行政院勞工委員會（以下簡稱勞委會）軟性凍結泰勞政策之決策過程是否有當，特向行政院提出質詢。               外籍勞工   
845    有鑑於近年來大客車（遊覽車）事故頻傳，屢屢造成重大傷亡，多肇因於車輛不符安全規格，建請行政院...  交通安全 ; 客運業 ; 車輛檢驗   
11021  為邱義仁自年輕時代打選戰至今，猶然樂此不疲，選戰幾乎成為他血液中的基本因子。惟他是否曾想到過...    選舉 ; 政務官 ; 行政中立   
4356   針對行政院金融監督管理委員會為了解決國人廣大的卡債問題，積極推廣債務協商機制，政策本意良好，...           信用卡 ; 債務   
825    有鑒於近日來交通部狀況不斷，從華航、高鐵、台鐵到公路意外事故頻傳，傳聞交通部擬盛大舉辦中元普...        意外事故 ; 行政效率   
1504   針對整合警力資源、擴大警政服務範疇，建議行政院將現行警政署、海岸巡防署、消防署及入出境管理局...               政府組織   
6636   針對「莊嚴肅穆」的行憲紀念日，在威權時代「今天應懸掛國旗」，不過，在愈來愈多人的真實生活中，...        紀念活動 ; 憲政運作   
5603   就我國經濟弊案，近8年起訴金額高達2,767億元，而重大經濟罪犯動輒潛逃出境，每判一個就逃一...         經濟犯罪 ; 脫逃罪   
9039   針對新聞局認定TVBS應為綜合台非新聞台乙案，準備將TVBS轉頻一事，日前行政院新聞局認定T...         新聞自由 ; 電視臺   
3525   對於政府要求台灣郵政公司每天挑選數萬封郵件蓋上台灣入聯的戳章一事，表示高度關切與嚴正的抗議！...        郵政機關 ; 政令宣導   
6532   針對陳總統日昨重申，有信心一定能夠在二○○八年卸任總統之前，為台灣催生一部合身、合時、合用的...               憲法修正   
5595   就立法委員選舉，改採單一選區兩票制即將首度實施，中央選舉委員應加強宣導「單一選區兩票制」的新...                 選舉   
716    針對新聞局長謝志偉於立院議場備詢時，被要求下台，卻堅不下台一事，顯示行政院對憲法之基本尊重全...          政務官 ; 立法院   
2339   針對近日新內閣多項工作狀況頻頻，引起治安、金融和外交上的政治危機，並嚴重影響政府、國家形象，...        內閣改組 ; 政府施政   
8839   針對市售豆類製品疑含「過氧化氫」情形嚴重，傷害消費者健康，爰要求相關單位依食品衛生管理法切實...               食品管理   

                                           Key Word  
1169                                           凍結泰勞  
845                             大客車 ; 遊覽車 ; 事故 ; 傷亡  
11021                         邱義仁 ; 國安會秘書長 ; 黨選戰操盤手  
4356                                      卡債 ; 協商機制  
825                                            意外事故  
1504                             警力資源 ; 警政服務 ; 警政總署  
6636                              聖誕節 ; 行憲紀念日 ; 制憲權  
5603                           經濟罪犯 ; 潛逃出境 ; 國際司法互助  
9039   TVBS ; 股權結構 ; 外資 ; 國家通訊傳播委員會組織法 ; NCC ; 報導自由  
3525                                  台灣郵政公司 ; 台灣入聯  
6532                                     新憲法 ; 修改憲法  
5595                               立法委員選舉 ; 單一選區兩票制  
716                                  新聞局長謝志偉 ; 立院議場  
2339                                             內閣  
8839                                    豆類製品 ; 過氧化氫

#### (2) Export an End-to-end Model


In [28]:
from tensorflow.keras.models import load_model

In [29]:
# Get model (Sequential, Functional Model, or Model subclass)
Dcnn.save('lour_pork_model')

INFO:tensorflow:Assets written to: lour_pork_model/assets


INFO:tensorflow:Assets written to: lour_pork_model/assets


In [30]:
from tensorflow import keras
model = keras.models.load_model('lour_pork_model')

import pandas as pd
import random

# test the trined model using new text from 200 samples from Leislative Questions
LQ5 = pd.read_csv('LQ_5th.csv')

# drop legislor whose speech token less than 15
LQ5 = LQ5[[len(x)>15 for x in LQ5.title]]

# split the sentence with space
full_set = [seg_char(text) for text in [clearPucts(text) for text in LQ5.title]]

def recode_as_pork(pork):
    if pork>0.75:
        return 1
    else:
        return 0

# fit the model    
LQ5['pork_value'] = pd.to_numeric([as_num(Dcnn(np.array([tokenizer.encode(line)]), training=True).numpy()[0][0]) for line in full_set])

# classify pork_value>0.75 into pork 
LQ5['pork_dummy'] = [recode_as_pork(x) for x in LQ5['pork_value']]

# save
LQ5.to_csv('LQ5_pork.csv', index=False)